# Detect Objects

- People + Trees
- Bus Stops + Signs

In [ ]:
import json

from os import listdir, makedirs, path
from PIL import Image as PImage, ImageDraw as PImageDraw

from utils.detect_utils import OBJECT_THRESHOLDS
from models.Dino import Dino
from models.Yolo import Yolo

In [ ]:
OBJ_DIR = "./data/objs"
IMG_DIR = "./imgs"
TECHNIQUES = ["address", "address_coords", "heading", "station_coords"]

## People + Trees

In [ ]:
OBJS = "people_trees"
model = Dino()

for t in TECHNIQUES:
  makedirs(path.join(OBJ_DIR, t, OBJS), exist_ok=True)
  fnames = sorted(f for f in listdir(path.join(IMG_DIR, t)) if f.endswith("jpg"))

  for idx,f in enumerate(fnames):
    if idx>1024 and idx%16==0: print(idx, "/", len(fnames))

    img_file = path.join(IMG_DIR, t, f)
    obj_file = path.join(OBJ_DIR, t, OBJS, f.replace(".jpg", ".json"))

    if path.isfile(obj_file): continue

    img = PImage.open(img_file)

    objs = model.iou_objects(img, OBJECT_THRESHOLDS)

    with open(obj_file, "w") as ofp:
      json.dump({ "dino" : objs }, ofp)

## Bus Stops

In [ ]:
OBJS = "bus_stop"

OBJECT_THRESHOLDS_YOLO = {
  "bus_stop": 0.25,
  "bus_sign": 0.25,
}

model = Yolo("./models/bus_20260201_yolo.pt")

for t in TECHNIQUES:
  makedirs(path.join(OBJ_DIR, t, OBJS), exist_ok=True)
  fnames = sorted(f for f in listdir(path.join(IMG_DIR, t)) if f.endswith("jpg"))

  for idx,f in enumerate(fnames):
    if idx>0 and idx%16==0: print(idx, "/", len(fnames))

    img_file = path.join(IMG_DIR, t, f)
    obj_file = path.join(OBJ_DIR, t, OBJS, f.replace(".jpg", ".json"))

    if path.isfile(obj_file): continue

    img = PImage.open(img_file)

    objs = model.all_objects(img, OBJECT_THRESHOLDS_YOLO)

    with open(obj_file, "w") as ofp:
      json.dump({ "yolo" : objs }, ofp)